In [9]:
!pip install -Uqq adapter-transformers datasets

%load_ext autoreload
%autoreload 1
%aimport adapter_utils
from adapter_utils import get_model, get_tokenizer, adapt_model, get_test_data

### Load and Adapt a Model
I've written helper functions to generalize / abstract the loading of the model and the tokenizer

In [13]:
tokenizer = get_tokenizer()

In [14]:
model = get_model()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [15]:
adapt_model(model=model, adapter_name="qa/squad1@ukp", adapter_arch="houlsby")

### Test that it's working
Test that things are working by running the Q&A example from Adapter Hub's [sample notebook](https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb).

In [16]:
from transformers import QuestionAnsweringPipeline

qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

context = """
The current modus operandi in NLP involves downloading and fine-tuning pre-trained models consisting of millions or billions of parameters.
Storing and sharing such large trained models is expensive, slow, and time-consuming, which impedes progress towards more general and versatile NLP methods that learn from and for many tasks.
Adapters -- small learnt bottleneck layers inserted within each layer of a pre-trained model -- ameliorate this issue by avoiding full fine-tuning of the entire model.
However, sharing and integrating adapter layers is not straightforward.
We propose AdapterHub, a framework that allows dynamic "stitching-in" of pre-trained adapters for different tasks and languages.
The framework, built on top of the popular HuggingFace Transformers library, enables extremely easy and quick adaptations of state-of-the-art pre-trained models (e.g., BERT, RoBERTa, XLM-R) across tasks and languages.
Downloading, sharing, and training adapters is as seamless as possible using minimal changes to the training scripts and a specialized infrastructure.
Our framework enables scalable and easy access to sharing of task-specific models, particularly in low-resource scenarios.
AdapterHub includes all recent adapter architectures and can be found at AdapterHub.ml.
"""

In [17]:
# ignore all FutureWarnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [18]:
def answer_questions(questions):
    for question in questions:
        result = qa(question=question, context=context)
        print("❔", question)
        print("💡", result["answer"])
        print()

answer_questions([
    "What are Adapters?",
    "What do Adapters avoid?",
    "What is proposed?",
    "What does AdapterHub allow?",
    "Where can I find AdapterHub?",
])

❔ What are Adapters?
💡 small learnt bottleneck layers

❔ What do Adapters avoid?
💡 full fine-tuning of the entire model.

❔ What is proposed?
💡 AdapterHub,

❔ What does AdapterHub allow?
💡 dynamic "stitching-in"

❔ Where can I find AdapterHub?
💡 AdapterHub.ml.



### List the datasets that exist at HuggingFace
HuggingFace makes it easy to access public NLP datasets

In [19]:
import datasets
datasets.list_datasets()[:20]

['acronym_identification',
 'ade_corpus_v2',
 'adversarial_qa',
 'aeslc',
 'afrikaans_ner_corpus',
 'ag_news',
 'ai2_arc',
 'air_dialogue',
 'ajgt_twitter_ar',
 'allegro_reviews',
 'allocine',
 'alt',
 'amazon_polarity',
 'amazon_reviews_multi',
 'amazon_us_reviews',
 'ambig_qa',
 'amttl',
 'anli',
 'app_reviews',
 'aqua_rat']

### Test some data
Test things with the rotten tomatoes example data

In [21]:
dataset = get_test_data()

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/jason/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5)
Loading cached processed dataset at /home/jason/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/e06abb624abab47e1a64608fdfe65a913f5a68c66118408032644a3285208fb5/cache-416f7cb970ff8c8c.arrow


### Test training
try the training procedure here: https://github.com/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb

In [23]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter
model.add_adapter("rotten_tomatoes")
# Add a matching classification head
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("rotten_tomatoes")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [24]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [25]:
trainer.train()

Step,Training Loss
200,0.491700
400,0.307200
600,0.278200
800,0.260900
1000,0.247800
1200,0.234400
1400,0.247900
1600,0.216000


TrainOutput(global_step=1602, training_loss=0.28541606363285793, metrics={'train_runtime': 120.3735, 'train_samples_per_second': 13.309, 'total_flos': 3098616289056000.0, 'epoch': 6.0})

In [26]:
trainer.evaluate()

{'eval_loss': 0.27985164523124695,
 'eval_acc': 0.8949343339587242,
 'eval_runtime': 0.889,
 'eval_samples_per_second': 1199.139,
 'epoch': 6.0}

In [27]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': '👍', 'score': 0.9930281043052673}]

In [28]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

!ls -lh final_adapter

total 5.7M
-rw-rw-r-- 1 jason jason  581 Jul  3 11:00 adapter_config.json
-rw-rw-r-- 1 jason jason  354 Jul  3 11:00 head_config.json
-rw-rw-r-- 1 jason jason 3.5M Jul  3 11:00 pytorch_adapter.bin
-rw-rw-r-- 1 jason jason 2.3M Jul  3 11:00 pytorch_model_head.bin


### The Arvix dataset is going to take more prep work
You have to manually download and extract this dataset, and even then I haven't been able to get it work yet. There's some nuance in the way you have to tell HuggingFace to split the data and I haven't gotten this part working yet.

In [6]:
from datasets import load_dataset, SplitInfo
arvix_data = load_dataset("arxiv_dataset", data_dir="./arvix_data/", split=SplitInfo(name='train', num_bytes=2246545603, num_examples=1796911, dataset_name='arxiv_dataset'))

Using custom data configuration default-data_dir=.%2Farvix_data%2F


NonMatchingSplitsSizesError: [{'expected': SplitInfo(name='train', num_bytes=2246545603, num_examples=1796911, dataset_name='arxiv_dataset'), 'recorded': SplitInfo(name='train', num_bytes=2403104294, num_examples=1905114, dataset_name='arxiv_dataset')}]